In [3]:
%%time

import cv2
import mediapipe as mp
import numpy as np
import os
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor
from functools import partial

def dealImg(img):
    """修正颜色转换函数"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
def blend_images(img1, img2, mask):
    """
    使用掩码混合两张图像
    参数:
        img1: 第一张图像 (numpy数组, uint8)
        img2: 第二张图像 (numpy数组, uint8)
        mask: 掩码图像 (numpy数组, float32, 值范围0-1)
    """
    # 确保掩码是单通道
    if len(mask.shape) == 3:
        mask = mask[:,:,0]  # 取第一个通道
    mask = np.expand_dims(mask, axis=2)  # 恢复为 [H,W,1]
    
    # 转换图像为float32
    img1 = img1.astype(np.float32)
    img2 = img2.astype(np.float32)
    
    # 混合图像
    blended = img1 * mask + img2 * (1 - mask)
    return np.clip(blended, 0, 255).astype(np.uint8)

def process_single_image(img_path, img_list, landmarker):
    try:
        # 1. 读取图像 (保持BGR格式)
        image = cv2.imread(img_path)
        if image is None:
            print(f"无法读取图像: {img_path}")
            return
        
        # 2. 转换为RGB供MediaPipe使用
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image_rgb)
        
        # 检测面部特征点
        face_landmarker_result = landmarker.detect(mp_image)
        if not face_landmarker_result.face_landmarks:
            print(f"未检测到面部: {img_path}")
            return
            
        landmarks = face_landmarker_result.face_landmarks[0]
        mp_height, mp_width = image.shape[:2]
        
        # 预计算常用坐标点
        left_up_x = int(landmarks[53].x * mp_width)
        left_up_y = int(landmarks[53].y * mp_height)
        right_down_x = int(landmarks[346].x * mp_width)
        right_down_y = int(landmarks[346].y * mp_height)
        eye_region_width = right_down_x - left_up_x
        eye_region_height = int(0.375 * eye_region_width)
        fix_ycenter = int((right_down_y + left_up_y) * 0.5)
        fix_left_up_y = fix_ycenter - int(0.5 * eye_region_height)
        
        # 创建眼部区域掩码
        mask = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.float32)
        left_eye_point = [33, 246, 161, 160, 159, 158, 157, 173, 133, 155, 154, 153, 145, 144, 163, 7]
        right_eye_point = [263, 466, 388, 387, 386, 385, 384, 398, 362, 382, 381, 380, 374, 373, 390, 249]
        
        left_eye_mask = []
        right_eye_mask = []
        for i in left_eye_point:
            left_eye_mask.append([int(landmarks[i].x * mp_width), int(landmarks[i].y * mp_height)])
        for i in right_eye_point:
            right_eye_mask.append([int(landmarks[i].x * mp_width), int(landmarks[i].y * mp_height)])
            
        lefteye_mask_temp = cv2.fillConvexPoly(mask, np.int32(left_eye_mask), (1.0, 1.0, 1.0))
        righteye_mask_temp = cv2.fillConvexPoly(mask, np.int32(right_eye_mask), (1.0, 1.0, 1.0))
        mask += lefteye_mask_temp
        mask += righteye_mask_temp
    
        mask = 255 * np.uint8(mask)
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40, 40))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, 1)
        mask = cv2.GaussianBlur(mask, (15, 15), cv2.BORDER_DEFAULT)
        
        inverseMask = cv2.bitwise_not(mask)
        mask = mask.astype(float)/255 
        inverseMask = inverseMask.astype(float)/255
        
        # 颜色映射处理（完全保留原始代码）
        lut = np.zeros((256,1,3), dtype=np.uint8)
        for i in range(256):
            if i == 0:
                lut[i] = (0,0,0)
            else:
                tmp = int(np.exp(np.log(i / 255.0) * 0.7) * 255.0)
                lut[i]=(tmp,tmp,tmp)
        eyes = cv2.LUT(image, lut)
        
        eyes = eyes.astype(float)/255 
        ladyFace = image.astype(float)/255
        
        justeyes = cv2.multiply(mask, eyes)
        justFace = cv2.multiply(inverseMask, ladyFace)
        first_result = justFace + justeyes
        first_result = np.clip(first_result, 0, 1)
        
        # 转换回0-255范围（新增）
        first_result = (first_result * 255).astype(np.uint8)
        
        # 第二步：面部模糊处理
        mask2 = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.float32)
        face_point = [10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 397, 365, 379, 378, 400, 377,
                     152, 148, 176, 149, 150, 136, 172, 58, 132, 93, 234, 127, 162, 21, 54, 103, 67, 109]
        
        face_mask = []
        for i in face_point:
            face_mask.append([int(landmarks[i].x * mp_width), int(landmarks[i].y * mp_height)])
     
        face_mask_temp = cv2.fillConvexPoly(mask2, np.int32(face_mask), (1.0, 1.0, 1.0))
        mask2 += face_mask_temp
        mask2[fix_left_up_y:fix_left_up_y + eye_region_height, left_up_x:left_up_x + eye_region_width] = 0
        mask2[fix_left_up_y + eye_region_height:int(landmarks[2].y*mp_height), int(landmarks[129].x*mp_width):int(landmarks[358].x*mp_width)] = 0
        
        # 优化模糊处理
        first_result_uint8 = first_result.astype(np.uint8)
        faces = cv2.medianBlur(first_result_uint8, 149)
        
        mask2_uint8 = (mask2 * 255).astype(np.uint8)
        mask2_uint8 = cv2.morphologyEx(mask2_uint8, cv2.MORPH_CLOSE, kernel, iterations=1)
        mask2_uint8 = cv2.GaussianBlur(mask2_uint8, (15, 15), 0)
        
        mask2_uint8 = (mask2 * 255).astype(np.uint8)
        mask2_float = mask2_uint8.astype(np.float32)/255
        second_result = blend_images(faces, first_result, mask2_float)
        
        # 第三步：特定区域模糊处理
        mask3 = np.zeros((mp_height, mp_width, 3), dtype=np.float32)
        mask3[fix_left_up_y + eye_region_height:int(landmarks[2].y * 1.05 * mp_height), 
              int(landmarks[129].x * 0.95*mp_width):int(landmarks[358].x * 1.05*mp_width)] = 1
        
        mask3_uint8 = (mask3 * 255).astype(np.uint8)
        mask3_uint8 = cv2.morphologyEx(mask3_uint8, cv2.MORPH_CLOSE, kernel, iterations=1)
        mask3_uint8 = cv2.GaussianBlur(mask3_uint8, (15, 15), 0)
        
        second_result_uint8 = second_result.astype(np.uint8)
        final_blur = cv2.medianBlur(second_result_uint8, 99)
        
        mask3_uint8 = (mask3 * 255).astype(np.uint8)
        mask3_float = mask3_uint8.astype(np.float32)/255
        final_result = blend_images(final_blur, second_result, mask3_float)
        
        # 第四步：裁切图片
        left_up_x = int(landmarks[234].x * mp_width)
        left_up_y = int(landmarks[10].y * mp_height)
        right_down_x = int(landmarks[454].x * mp_width)
        right_down_y = int(landmarks[152].y * mp_height)
        face_width = right_down_x - left_up_x
        face_height = right_down_y - left_up_y
        
        top = max(left_up_y - int(0.6 * face_height), 0)
        bottom = min(right_down_y + int(0.25 * face_height), mp_height)
        left = max(left_up_x - int(0.25 * face_width), 0)
        right = min(right_down_x + int(0.25 * face_width), mp_width)
        
        cropped_result = final_result[top:bottom, left:right]
        
        # 保存结果 - 修正部分
        output_dir = os.path.join('features_blur', img_list)
        os.makedirs(output_dir, exist_ok=True)
        
        # 从img_path提取纯文件名
        filename = os.path.basename(img_path)
        output_path = os.path.join(output_dir, filename)
        cv2.imwrite(output_path, cropped_result)  # 直接保存BGR格式
        
    except Exception as e:
        print(f"处理图像 {img_path} 时出错: {str(e)}")

# 主函数
def main():
    path = './'
    model_path = 'face_landmarker.task'
    
    # 初始化MediaPipe
    BaseOptions = mp.tasks.BaseOptions
    FaceLandmarker = mp.tasks.vision.FaceLandmarker
    FaceLandmarkerOptions = mp.tasks.vision.FaceLandmarkerOptions
    VisionRunningMode = mp.tasks.vision.RunningMode
    
    options = FaceLandmarkerOptions(
        base_options=BaseOptions(model_asset_path=model_path),
        running_mode=VisionRunningMode.IMAGE,
        num_faces=1)
    
    landmarker = FaceLandmarker.create_from_options(options)
    
    # 获取图片列表
    imgs = [i for i in os.listdir(path) if '2' in i]
    
    # 使用多线程处理
    with ThreadPoolExecutor(max_workers=4) as executor:  # 根据CPU核心数调整
        for img_list in imgs:
            img_dir = os.path.join(path, img_list)
            if not os.path.isdir(img_dir):
                continue
                
            img_files = [os.path.join(img_dir, f) for f in os.listdir(img_dir) 
                        if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
            
            # 批量处理每个目录中的图片
            executor.map(partial(process_single_image, img_list=img_list, landmarker=landmarker), img_files)
    
    landmarker.close()

if __name__ == "__main__":
    main()

CPU times: total: 1.22 s
Wall time: 1.25 s
